In [ ]:
import os
import numpy as np
from google import genai
from google.genai import types
from dotenv import load_dotenv
from lightrag.utils import EmbeddingFunc
from lightrag import LightRAG, QueryParam
from lightrag.kg.shared_storage import initialize_pipeline_status
from sentence_transformers import SentenceTransformer

import asyncio
import nest_asyncio
import json
nest_asyncio.apply()

# DIR to store the index files
WORKING_DIR = "./data_with_bge_embedding"

load_dotenv()
gemini_key = os.getenv("GOOGLE_API_KEY")

d:\hehe\custom_lib\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # refresh foulder
# if os.path.exists(WORKING_DIR):
#     import shutil
#     shutil.rmtree(WORKING_DIR)
# os.mkdir(WORKING_DIR)

In [3]:
client = genai.Client(api_key=gemini_key)
sentences_embedding = SentenceTransformer("dangvantuan/vietnamese-document-embedding", trust_remote_code=True)
sentences_embedding.max_seq_length = 1024


async def llm_model_func(prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs) -> str:
    if history_messages is None:
        history_messages = []
    combined_prompt = ""
    if system_prompt:
        combined_prompt += f"{system_prompt}\n"
    for msg in history_messages:
        combined_prompt += f"{msg['role']}: {msg['content']}\n"

    combined_prompt += f"user: {prompt}"

    response = client.models.generate_content(
        model="gemini-1.5-flash",
        contents=[combined_prompt],
        config=types.GenerateContentConfig(max_output_tokens=500, temperature=0.1),
    )
    return response.text


async def embedding_func(texts: list[str]) -> np.ndarray:
    embeddings = sentences_embedding.encode(texts, convert_to_numpy=True)
    return embeddings


async def initialize_rag():
    rag = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=llm_model_func,
        embedding_func=EmbeddingFunc(
            embedding_dim=768,
            max_token_size=8192,
            func=embedding_func,
        ),
    )

    await rag.initialize_storages()
    await initialize_pipeline_status()
    return rag

rag = asyncio.run(initialize_rag())

INFO: Process 10944 Shared-Data created for Single Process
INFO:nano-vectordb:Load (2723, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './data_with_bge_embedding\\vdb_entities.json'} 2723 data
INFO:nano-vectordb:Load (478, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './data_with_bge_embedding\\vdb_relationships.json'} 478 data
INFO:nano-vectordb:Load (480, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './data_with_bge_embedding\\vdb_chunks.json'} 480 data
INFO: Process 10944 initialized updated flags for namespace: [full_docs]
INFO: Process 10944 ready to initialize storage namespace: [full_docs]
INFO: Process 10944 initialized updated flags for namespace: [text_chunks]
INFO: Process 10944 ready to initialize storage namespace: [text_chunks]
INFO: Process 10944 initialized updated flags for namespace: [entities]
INFO: Process 10944 initialized 

In [ ]:
import time
import json
import logging

# Set the logging level to WARNING for specific libraries
logging.getLogger("sentence_transformers").setLevel(logging.WARNING)
logging.getLogger("google_genai").setLevel(logging.WARNING)

with open('documents.json', 'r', encoding='utf-8') as f:
    documents = json.load(f)  
documents = list(documents.values())
    
idx = 00 
while idx < len(documents):
    doc = documents[idx]
    try:
        rag.insert(doc)
        print(f"Inserted document {idx}")
        idx += 1 
    except Exception as e:
        print(f"Error inserting document {idx}: {e}")
        time.sleep(40)  

Inserted document 475
Inserted document 476
Inserted document 477
Inserted document 478
Inserted document 479
Inserted document 480
Inserted document 481
Inserted document 482
Inserted document 483
Inserted document 484
Inserted document 485
Inserted document 486
Inserted document 487
Inserted document 488
Inserted document 489
Inserted document 490
Inserted document 491
Inserted document 492
Inserted document 493
Inserted document 494
Inserted document 495
Inserted document 496
Inserted document 497
Inserted document 498
Inserted document 499
Inserted document 500
Inserted document 501
Inserted document 502
Inserted document 503
Inserted document 504
Inserted document 505
Inserted document 506
Inserted document 507
Inserted document 508
Inserted document 509
Inserted document 510
Inserted document 511
Inserted document 512
Inserted document 513
Inserted document 514
Inserted document 515
Inserted document 516
Inserted document 517
Inserted document 518
Inserted document 519
Inserted d

Failed to extract entities and relationships
Failed to process document doc-69060013983d2e2f2d7ac61dbc9eb90b: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The service is currently unavailable.', 'status': 'UNAVAILABLE'}}


Inserted document 594
Inserted document 595
Inserted document 596
Inserted document 597
Inserted document 598
Inserted document 599
Inserted document 600
Inserted document 601
Inserted document 602
Inserted document 603
Inserted document 604
Inserted document 605
Inserted document 606
Inserted document 607
Inserted document 608
Inserted document 609
Inserted document 610
Inserted document 611
Inserted document 612
Inserted document 613
Inserted document 614
Inserted document 615
Inserted document 616
Inserted document 617
Inserted document 618
Inserted document 619
Inserted document 620
Inserted document 621
Inserted document 622
Inserted document 623
Inserted document 624
Inserted document 625
Inserted document 626
Inserted document 627
Inserted document 628
Inserted document 629
Inserted document 630
Inserted document 631
Inserted document 632
Inserted document 633
Inserted document 634
Inserted document 635
Inserted document 636
Inserted document 637
Inserted document 638
Inserted d

In [ ]:
# thêm địa chỉ trường học vào graph
import json
with open('addresses.json', 'r', encoding='utf-8') as f:
    document = json.load(f)
    
geos = list(set(document.values()))
mien_bac = ['Hà Nội', 'Hải Phòng', 'Bắc Ninh', 'Thái Nguyên', 'Thái Bình', 'Hải Dương', 'Vĩnh Phúc']
mien_trung = ['Huế', 'Thừa Thiên Huế', 'Đà Nẵng', 'Thanh Hóa', 'Nghệ An', 'Khánh Hòa', 'Đà Lạt']
mien_nam = ['Thành phố Hồ Chí Minh', 'Cần Thơ', 'Đồng Nai', 'Bình Dương', 'Long An', 'Hưng Yên']

def locate_geo(geo):
    if geo in mien_bac:
        return "Miền Bắc"
    elif geo in mien_trung:
        return "Miền Trung"
    elif geo in mien_nam:
        return "Miền Nam"
    else:
        return None
for locate in ["Miền Bắc", "Miền Trung", "Miền Nam"]:
    try:
        entity = rag.create_entity(locate, { 
        "description": f"{locate} là một miền của Việt Nam.",
        "entity_type": "geo",
        })
    except Exception as e:
        print(f"Error creating entity for {locate}: {e}")

for geo in geos:
    try:
        entity = rag.create_entity(geo,{ 
        "description": f"{geo} là một tỉnh thuộc {locate_geo(geo)} của Việt Nam.",
        "entity_type": "geo",
        })
        
        relation = rag.create_relation(geo, locate_geo(geo), {
        "description": f"{geo} là một tỉnh thuộc {locate_geo(geo)} của Việt Nam.",
        "keywords": f"{locate_geo(geo)}, {geo}, tỉnh, thành phố",
        "weight": 10.0
        })
    except Exception as e:
        print(f"Error creating entity for {geo}: {e}")
                
for school, geo in document.items():
    try:
        relation = rag.create_relation(school, geo, {
        "description": f"{school} là một trường học tại {geo}.",
        "keywords": f"{geo}, {school}, tỉnh, thành phố",
        "weight": 10.0
        })
    except Exception as e:
        print(f"Error creating relation for {school} {geo}: {e}")

In [12]:
response = rag.query(
    query= "điểm các ngành của trường đại học bách khoa hà nội",
    param=QueryParam(mode="mix", top_k=5, response_type="single line", only_need_context=False),
)

print(response)

Điểm chuẩn năm 2024 của Đại học Bách khoa Hà Nội: Công nghệ thông tin (Khoa học máy tính: 28,53; Kỹ thuật máy tính: 28,48; Khoa học dữ liệu và Trí tuệ nhân tạo: 28,22); Điện - Điện tử (Kỹ thuật điện tử viễn thông: 27,12; Kỹ thuật điều khiển và tự động hóa: 27,45; Kỹ thuật điện: 25,85); Cơ khí (Kỹ thuật cơ khí: 25,75; Kỹ thuật cơ điện tử: 26,55; Kỹ thuật ô tô: 26,25); Hóa học (Kỹ thuật hóa học: 24,38; Hóa học: 23,81; Kỹ thuật sinh học: 24,00); Các ngành khác (Vật lý kỹ thuật: 23,50; Kỹ thuật vật liệu: 22,80; Quản lý công nghiệp: 25,90; Kinh tế công nghiệp: 26,15); Chương trình tiên tiến (Kỹ thuật thực phẩm: 22,00; Kỹ thuật sinh học: 22,00; Kỹ thuật Hóa dược: 24,34); Chương trình liên kết quốc tế (Quản trị Kinh doanh (TROY-BA): 21,00; Khoa học Máy tính (TROY-IT): 21,00).

